# First Experiences with MongoDB and BaseX
---------------

In [3]:
#!pip install pymongo

  Using cached pymongo-4.6.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
  Using cached dnspython-2.4.2-py3-none-any.whl.metadata (4.9 kB)
Using cached pymongo-4.6.1-cp311-cp311-win_amd64.whl (472 kB)
Using cached dnspython-2.4.2-py3-none-any.whl (300 kB)


In [6]:
# Import libraries
import json
from pymongo import MongoClient

In [7]:
# Connect with MongoDB server
client = MongoClient('localhost', 27017)
print("Connection Successful")

Connection Successful


In [8]:
# Create Database called data 
db = client['data']

# Create database called 'restaurants'
restaurants = db['restaurants']

In [9]:
# Initialize an empty list to store data
data = []

# Open the restaurant file containing the data in JSON format
with open('restaurants_mongodb.json', 'r') as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")

# Insert the data into the collection
if data:
    restaurants.insert_many(data)
    print("Data imported into MongoDB successfully.")
else:
    print("No data to import.")


Data imported into MongoDB successfully.


##### Total number of documents in the collection 'restaurants'

In [10]:
total_documents = restaurants.count_documents({})
print("Total number of documents:", total_documents)

Total number of documents: 25359


##### First 5 restaurants names with 'Bronx' as borough from the collection 'restaurants'

In [11]:
cursor = restaurants.find({"borough": "Bronx"}).limit(5)
for document in cursor:
    print('Name :', document['name'], 'Restaurant ID: ', document['restaurant_id'], 'Borough: ',document['borough'])

Name : Morris Park Bake Shop Restaurant ID:  30075445 Borough:  Bronx
Name : Wild Asia Restaurant ID:  40357217 Borough:  Bronx
Name : Carvel Ice Cream Restaurant ID:  40363093 Borough:  Bronx
Name : Happy Garden Restaurant ID:  40363289 Borough:  Bronx
Name : Happy Garden Restaurant ID:  40364296 Borough:  Bronx


##### Restaurants with a score higher than 80 but less than 100

In [16]:
high_score_restaurants = restaurants.find({"grades": {"$elemMatch": {"score": {"$gt": 80, "$lt": 100}}}})
count = restaurants.count_documents({"grades": {"$elemMatch": {"score": {"$gt": 80, "$lt": 100}}}})
print('Number of Restaurants with a score between 80 and 100:' ,count)

for restaurant in high_score_restaurants:
    print(restaurant["name"], restaurant['borough'])
    for grade in restaurant['grades']:
        if 80 < grade['score'] < 100:
            print("Score:", grade['score'])

Number of Restaurants with a score between 80 and 100: 11
Gandhi Manhattan
Score: 92
Bella Napoli Manhattan
Score: 98
West 79Th Street Boat Basin Cafe Manhattan
Score: 89
Spicy Shallot Queens
Score: 84
Bistro Caterers Manhattan
Score: 84
Concrete Restaurant Manhattan
Score: 90
Anella Brooklyn
Score: 81
Baluchi'S Indian Food Manhattan
Score: 98
Cafe R Manhattan
Score: 82
D & Y Restaurant Brooklyn
Score: 86
La Potencia Restaurant Bronx
Score: 82


##### Name of the restaurants which contain 'Wil' as first three letters in its name.


In [13]:
wil_restaurants = restaurants.find({"name": {"$regex": "^Wil"}})
count = restaurants.count_documents({"name": {"$regex": "^Wil"}})
print("Number of restaurats which contain 'Wil' in the first three letters", count)

for restaurant in wil_restaurants:
    print(restaurant["name"], restaurant['borough'])


Number of restaurats which contain 'Wil' in the first three letters 31
Wilken'S Fine Food Brooklyn
Wild Asia Bronx
Wilbel Pizza Bronx
Wild Edibles Manhattan
Wild Bagels Brooklyn
Willie'S Steak House Bronx
Willburg Cafe Brooklyn
Willis North America Manhattan
William Greenberg Jr Desserts Manhattan
Wild Ginger Vegan Cafe Brooklyn
Wilfie & Nell Manhattan
Wild Ginger Vegetarian Kitchen Manhattan
Wild Manhattan
William Barnacle Manhattan
William Hallet Queens
Williams & Bailey Brooklyn
Wild Ginger Manhattan
William Greenberg Dessert Manhattan
Williamsburg Pizza Brooklyn
Wild Cherry Queens
Wilmer Hale Manhattan
Willow Road Manhattan
Wilma Jean Brooklyn
Williamsburg Cinemas Brooklyn
Williams Burgers Queens
Williamsburg Creamery Brooklyn
Williamsburg Pizza Manhattan
Williamsbridge Diner Bronx
Wild Orchid Bar & Lounge Queens
Willie O'S Queens
Wild Ginger Brooklyn


##### Restaurant names which are not belonging to the borough Staten Island or Queens or Bronx or Brooklyn.


In [18]:
not_in_boroughs = ["Staten Island", "Queens", "Bronx", "Brooklyn"]

count = restaurants.count_documents({"borough": {"$nin": not_in_boroughs}})
print("Number of Restaurants not in the speciefied list:", count)

restaurants_not_in_specified_boroughs = restaurants.find({"borough": {"$nin": not_in_boroughs}})

for restaurant in restaurants_not_in_specified_boroughs:
    print('Name: ', restaurant["name"], 'Borough: ', restaurant['borough'])


Number of Restaurants not in the speciefied list: 10310
Name:  Dj Reynolds Pub And Restaurant Borough:  Manhattan
Name:  1 East 66Th Street Kitchen Borough:  Manhattan
Name:  Glorious Food Borough:  Manhattan
Name:  Bully'S Deli Borough:  Manhattan
Name:  Harriet'S Kitchen Borough:  Manhattan
Name:  P & S Deli Grocery Borough:  Manhattan
Name:  Angelika Film Center Borough:  Manhattan
Name:  The Country Cafe Borough:  Manhattan
Name:  Downtown Deli Borough:  Manhattan
Name:  Olive'S Borough:  Manhattan
Name:  Cafe Metro Borough:  Manhattan
Name:  Lexler Deli Borough:  Manhattan
Name:  Lorenzo & Maria'S Borough:  Manhattan
Name:  Domino'S Pizza Borough:  Manhattan
Name:  Berkely Borough:  Manhattan
Name:  Domino'S Pizza Borough:  Manhattan
Name:  Spoon Bread Catering Borough:  Manhattan
Name:  Texas Rotisserie Borough:  Manhattan
Name:  Metropolitan Club Borough:  Manhattan
Name:  Palm Restaurant Borough:  Manhattan
Name:  21 Club Borough:  Manhattan
Name:  Isle Of Capri Resturant Borou

##### Each borough and the number of restaurants for each borough


In [19]:
pipeline = [
    {
        "$group": {
            "_id": "$borough",
            "count": {"$sum": 1}
        }
    }
]

borough_counts = restaurants.aggregate(pipeline)

for entry in borough_counts:
    print(f"Borough: {entry['_id']}, Number of Restaurants: {entry['count']}")


Borough: Manhattan, Number of Restaurants: 10259
Borough: Bronx, Number of Restaurants: 2338
Borough: Brooklyn, Number of Restaurants: 6086
Borough: Queens, Number of Restaurants: 5656
Borough: Staten Island, Number of Restaurants: 969
Borough: Missing, Number of Restaurants: 51


##### Restaurant with the term 'Mexican' in the name attribute but don't have mexican as content in the cuisine attribute.


In [20]:
mexican_name_not_cuisine = restaurants.find({"name": {"$regex": "Mexican", "$options": "i"}, "cuisine": {"$ne": "Mexican"}})
count = restaurants.count_documents({"name": {"$regex": "Mexican", "$options": "i"}, "cuisine": {"$ne": "Mexican"}})
print('Number of restaurants which have "Mexican" in the name but not in the cuisine attribute:', count)
for restaurant in mexican_name_not_cuisine:
    print(restaurant["name"],restaurant["cuisine"], restaurant['restaurant_id'])


Number of restaurants which have "Mexican" in the name but not in the cuisine attribute: 34
El Azteca Mexican Restaurant Latin (Cuban, Dominican, Puerto Rican, South & Central American) 40386927
Chipotle Mexican Grill Tex-Mex 41048429
Chipotle Mexican Grill Tex-Mex 41084776
Chipotle Mexican Grill Tex-Mex 41098952
Chipotle Mexican Grill Tex-Mex 41098956
Chipotle Mexican Grill Tex-Mex 41137141
Chipotle Mexican Grill Tex-Mex 41155290
El Bravo Mexican Bar & Restaurant Latin (Cuban, Dominican, Puerto Rican, South & Central American) 41311234
La Parranda Mexicana Bar & Restaurant Bottled beverages, including water, sodas, juices, etc. 41331535
Venga Fresh Mexican Grill Southwestern 41389872
Chipotle Mexican Grill Tex-Mex 41453954
Tong Xing Chinese Restaurant & New Taco Mexican Chinese 41531467
Chipotle Mexican Grill #1394 Tex-Mex 41537945
Cantina Mexicana Spanish 41582708
El Acatlan Mexican Restaurant Tex-Mex 41596374
Au Bon Pain/ Andale Mexican Grill (Bxcc) American  41619215
Qdoba Mexican 

##### Names of bakeries with 'sugar' in the name.

In [21]:
sugar_bakeries = restaurants.find({"name": {"$regex": "sugar", "$options": "i"}, "cuisine": "Bakery"})
for bakery in sugar_bakeries:
    print(bakery["name"])

Sugar Bun Bakery
Sugar City Bakery
Sugar Sweet Sunshine
Sugar Couture


#### BaseX
The same tasks as in the first Part of the Script but in BaseX

 
(1)
 
`count(//restaurant__id)`

**25359 Results**

***
 
(2)
 
```
for $restaurant in subsequence(//element()[borough = "Bronx"], 1, 5) 
return $restaurant/name/string()
```

**Output:**

Morris Park Bake Shop

Wild Asia

Carvel Ice Cream

Happy Garden

Happy Garden



 ***
(3)

**Our initial approach:**
```
for $score in /json/*/grades/*/score
where $score castable as xs:double and data($score)>80 and data($score) <100
return $score/../../../data(name)
```
**Approach from the lecture:**
```
for $restaurant in collection('restaurants_basex')//_
let $score := $restaurant/grades/_/score/text()
where $score > 80 and $score < 100
return $restaurant/name
```
**Output:**
*11 Results*

Gandhi

Bella Napoli

West 79Th Street Boat Basin Cafe

Spicy Shallot

Bistro Caterers

Concrete Restaurant

Anella

Baluchi'S Indian Food

Cafe R

D & Y Restaurant

La Potencia Restaurant

***

(4)
```
for $restaurant in //element()
where matches($restaurant/name, "^Wil")
return
  <result>
    <name>{$restaurant/name/string()}</name>
    <borough>{$restaurant/borough/string()}</borough>
  </result>
```

**31 Results** 
***

(5)
```
for $restaurant in collection('restaurants_basex')//_[not(borough = ('Staten Island', 'Queens', 'Bronx', 'Brooklyn'))]
return $restaurant/name
```

**10'310 results**
***
(6)
Both options are possible here:
```
for $borough in distinct-values(//borough/data())
let $count := count(//element()[borough/data() = $borough])
return
  <result>
    <borough>{$borough}</borough>
    <count>{$count}</count>
  </result>
```

```
for $restaurant in //element()[borough/data()]
group by $borough := $restaurant/borough/data()
return
  <result>
    <borough>{$borough}</borough>
    <count>{count($restaurant)}</count>
  </result>
```

**Output:** 

Bronx: 2338

Brooklyn: 6086

Manhattan: 10259

Queens: 5656

Staten Island: 969

Missing: 51

***
(7)
``` 
for $restaurant in //element()
where contains($restaurant/name/data(), "Mexican") and not(contains($restaurant/cuisine/data(), "Mexican"))
return
  <result>
    <name>{$restaurant/name/data()}</name>
  </result>
``` 
**34 Results**
***
(8)
``` 
for $restaurant in //element()
where contains(lower-case($restaurant/name/data()), "sugar") and $restaurant/cuisine/data() = "Bakery"
return
  $restaurant/name/data()
```
**Output:**

Sugar Bun Bakery

Sugar City Bakery

Sugar Sweet Sunshine

Sugar Couture

#### Conclusions 

I found out that MongoDB is more suited for handling JSON files. I think this is because MongoDB is NoSQL based. Furthermore, MongoDB allowed me to retrieve information more effectively than BaseX. BaseX is suited for XML files, which makes handling more difficult compared to MongoDB.

The query language of MongoDB is more intuitive if you're familiar with JSON files. I had bigger problems writing the query for BaseX, and it required a deeper understanding to receive the same results.

I think that the choice of whether to use MongoDB or BaseX comes down to the specific usage. In my case, I found that MongoDB suited better to my needs because I was handling JSON files. On the other hand, I also think that BaseX would be better with complex XML files.

Thanks to the larger MongoDB community, it was easier for me to find the right solution for the problems I encountered during the project. When solving problems for BaseX, I spent a lot of time doing research.

In the end, I can say that both MongoDB and BaseX have their strengths. I preferred MongoDB because it was more intuitive for me than BaseX, and also because the provided documentation was better than that of BaseX.